### Loading packages

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os.path
import glob

#modify some matplotlib parameters to manage the images for illustrator
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

In [ ]:
from skimage.filters import gaussian
import skimage.feature as skfeat

### Loading images

In [ ]:
fpath = os.path.abspath('test_images/R1 24h/')
print(fpath+ '\n')

im_names = file_names(fpath,'tiff')
im_count = len(im_names)
im_fnames = []

for name in im_names:
    im_fnames.append(os.path.join(fpath, name))

print('folder "' +fpath.split('\\')[-1]+'" = '+str(im_count) + ' files' + '\n')
print('file names:')
print(im_names)

In [ ]:
#exploring images features...
plt.imread(im_fnames[0]).shape

In [ ]:
im_data={}
im_data['R'],im_data['G'],im_data['B'] = get_im_data(im_fnames)
im_data['name']=im_fnames     # to store the related image source

In [ ]:
# also define a vector with the channels
CHANNELS=['R','G','B']

In [ ]:
im_number = 0
plot_channels(im_data, im_number)
im_number = 1
plot_channels(im_data, im_number)
im_number = 2
plot_channels(im_data, im_number)

Accord above images, all the dyes have signal in the Green channel and have one without signal.
Also, biggest signal per dye are: Dapi = B, Dii = R, Dio = G

In [ ]:
plt.figure(figsize=(12,8))
plt.imshow(im_data['B'][:,:,0])
plt.title('original')
plt.colorbar()

thr = 200 
plt.figure(figsize=(12,8))
plt.imshow(im_data['B'][:,:,0]>thr)
plt.title('threshold = '+ str(thr))

thr = 250
plt.figure(figsize=(12,8))
plt.imshow(im_data['B'][:,:,0]>thr)
plt.title('threshold = '+ str(thr))

In [ ]:
plt.figure(figsize=(12,8))
plt.imshow(im_data['R'][:,:,1])
plt.title('original')
plt.colorbar()

thr = 170 
plt.figure(figsize=(12,8))
plt.imshow(im_data['R'][:,:,1]>thr)
plt.title('threshold = '+ str(thr))

thr = 250
plt.figure(figsize=(12,8))
plt.imshow(im_data['R'][:,:,1]>thr)
plt.title('threshold = '+ str(thr))

In [ ]:
select_area(500, 1000, 500, 1000, im_data, 'R', 1)

In [ ]:
plt.figure(figsize=(12,8))
plt.imshow(im_data['R'][500:1000,500:1000,1])#,cmap='gray')
plt.title('original')
plt.colorbar()

In [ ]:
thr=70

colony_blobs_id(im_data['R'][:,:,1], thr, im_fnames[1], [2,5], xlim=[500,1000], ylim=[500,1000])  #The radius of each blob is approximately √2σ
#0.8 means a radius = 1 px.
#1px = 0,177um
# 1 cell 1 -2 um ---> min radius = 0.5 um = 2.82 px --> σ = 2.82/√2 = 1.994 px

In [ ]:
from skimage import data, feature, exposure

In [ ]:
img = data.coins()
plt.imshow(img)

In [ ]:
imge = exposure.equalize_hist(img)
plt.imshow(imge)

In [ ]:
feature.blob_log(img, threshold = .3)

## Functions

In [ ]:
def file_names(path,file_type):
    """
    To obtain a list with the name of the files with defined extension (filetype) on a certain folder (path)

    Parameters
    ----------
    path : string
        folder name where the images are stored

    file_type : string
        extension of the files to search for (e.g. tif, png, jpg)

    Returns
    -------
    F_Names : list
        number of defined filetype files on the path folder

    """
    
    F_Names = []
    for name in os.listdir(path):
        if name.find(file_type) != -1:
            F_Names.append(name)
            
    return(F_Names)


In [ ]:
def get_im_data(f_names):
    """
    Load image data from a list of image files (f_names).
    It doesn't supports images with different pixel dimentions
    (e.g all of them have to be 900x600 px)
    It is important for further positional correlations

    Parameters
    ----------
    f_name : string
        file name including full path where images are stored, e.g. "/folder/image_1.jpg"

    Returns
    -------
    ImsR,ImsG,ImsB: array_like
        data per channel of each image (ImsR -> matrix size = (W,H,image_count/x_frames))

    """
    
    W,H,_ = plt.imread(f_names[0]).shape      # Measure the image size based on the first image on the folder
    N = len(f_names)
    Ims_R = np.zeros((W,H,N))
    Ims_G = np.zeros((W,H,N))
    Ims_B = np.zeros((W,H,N))
    
    for i in range(N):
        Im = plt.imread(f_names[i])
        Ims_R[:,:,i] = Im[:,:,0]              # Last number code the channel: 0=red, 1=green, 2=blue
        Ims_G[:,:,i] = Im[:,:,1]
        Ims_B[:,:,i] = Im[:,:,2]
    return(Ims_R,Ims_G,Ims_B)

# at call you can take only the channels you are interested in (e.g.):
# red,_,blue=get_im_data(f_names)  ---> this only takes the red and blue channels



In [ ]:
def plot_channels(data, image_number):

    """
    plot channels of a desired image data in the dictionary

    Parameters
    ----------
    data: dictionary
        4 dimensional (R,G,B, im_file_number) matrix with the data 
    
    image_number: int
        number of the image in the array data

    Returns
    -------


    """
    
    plt.figure(figsize=(16,3))
    POS_VECT = [131,132,133]           # figure position vector
    count = 0
    plt.suptitle(data['name'][image_number].split('\\')[-1], fontsize=14)
    #print(data['name'][image_number].split('\\')[-1])
    
    for c in CHANNELS:

        # make plot of the sum over time of smoothed data per channel
    
        plt.subplot(POS_VECT[count])
        plt.imshow(data[c][:,:,image_number])
        plt.colorbar()
        plt.title(c+' channel')
    
        count += 1
    plt.subplots_adjust(hspace=0, wspace=0, top=0.8)

    return()

In [ ]:
def colony_blobs_id(data, thresh, im_name, sigma_lim=[1,10], filename='null', xlim='null', ylim='null'):
    """
    Use skimage to identify the position of each colony and define the circular region
    used by each of them

    Parameters
    ----------
    data: array of single channel image data

    thresh:
        Pixel values > thresh are included in the analysis, range (0,1)
        
    im_name:
        Name of an image on which to overlay colony positions and sizes
    
    filename: string
        filename with whom save the output image+blobs+ID
    
    sigma_lim: list
        list with lower and upper sigma limits [min_sigma, max_sigma]

    Returns
    -------
    A: array (Nx3)
        Contains the (x,y) position and size of each blob for each of N colonies detected
    """
    if xlim != 'null' and ylim != 'null':
        data = data[xlim[0]:xlim[1],ylim[0]:ylim[1]]
        
    A = skfeat.blob_log(data, min_sigma=sigma_lim[0], max_sigma=sigma_lim[1], num_sigma=10, 
                        threshold=thresh, overlap=0.5)

    plt.figure(figsize=(8,8))
    plt.imshow(data, cmap='gray')
    #plt.hold(True)
    plt.title('Sumarized Image')
    for i in range(len(A)):
        circle = plt.Circle((A[i,1], A[i,0]), 2*A[i,2], color='r', fill=False , 
                            lw=0.5)
        fig = plt.gcf()
        ax = fig.gca()
        ax.add_artist(circle)

    plt.figure(figsize=(8,8))
    if xlim != 'null' and ylim != 'null':
        plt.imshow(plt.imread(im_name)[xlim[0]:xlim[1],ylim[0]:ylim[1],:])
    else:
        plt.imshow(plt.imread(im_name))
    #plt.hold(True)
    plt.title('Over '+ im_name)
    for i in range(len(A)):
        # plot the circle area identified for each colony
        circle = plt.Circle((A[i,1], A[i,0]), 2*A[i,2], color='w', fill=False , lw=0.5)
        fig = plt.gcf()
        ax = fig.gca()
        ax.add_artist(circle)
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
        
        # attach the ID label to each colony
        plt.annotate(i, xy=(A[i,1], A[i,0]), xytext=(-2, 2),
                     textcoords='offset points', ha='right', va='bottom',
                     color='white')
    if filename != 'null':
        plt.savefig(str(filename) + ".pdf", transparent=True)

    return(A)


In [ ]:
def smooth_data(data,sigma, im_number):

    """
    Apply gaussian filter to smooth each frame data

    Parameters
    ----------
    data: dictionary
        4 dimensional (R,G,B, and im_number) matrix with the data.
        
    sigma: double
        Filter parameter (standard deviation)

    Returns
    -------
    NSIms: dictionary
        Sum over time of Smoothed data per channel (call it nsims[channel][r,c])

    NSImsAll: array_like
        Matrix with sum of nsims over the channels (call it nsimsAll[r,c])
    
    SImsT: dictionary
        Smoothed data per channel per frame (call it as simsT[channel][r,c,f])

    """

    NSIms = {}
    NSIms_All = np.zeros((data[CHANNELS[0]].shape[0],
                          data[CHANNELS[0]].shape[1]))
    SImsT = {}
    
    plt.figure(figsize=(17,3))
    POS_VECT = [131,132,133]           # figure position vector
    count = 0

    for c in CHANNELS:
        # apply filter
        #Data_Sum = data[c].sum(axis=2)
        SIms = gaussian(data[c][:,:,im_number], sigma)
        NSIms [c] = (SIms-SIms.min())/(SIms.max()-SIms.min())  # nomalization, pixel value ∈ [0,1]

        NSIms_All += NSIms[c]
        
        Maux = np.zeros((data[CHANNELS[0]].shape))
        for fr in range(data[c].shape[-1]):
            Maux[:,:,fr] = gaussian(data[c][:,:,fr], sigma) 
            
        SImsT[c] = Maux
        # make plot of the sum over time of smoothed data per channel
    
        plt.subplot(POS_VECT[count])
        plt.imshow(NSIms[c])
        plt.colorbar()
        plt.title(c+' channel')
    
        count += 1
    
    return(NSIms,NSIms_All,SImsT)

In [ ]:
def select_area(x1, x2, y1, y2, data, channel, im_num):
    """
    compute the background mean value for each channel and frame based on a rectagle
    defined by the user. Plot the rectangle over the image and makes plots of each channel
    mean background value over time

    Parameters
    ----------
    x1,x2,x1,x2: int values
        rectangle area limits: (x1,y1) = left-up corner. (x2,y2) = rigth-bottom corner
    
    data : dictionary
        R G B images data to get the background, and his names on data['Im']
    
    im_count : int
        total number of files on the folder (can be obtained with count_files function)

    Returns
    -------
    bg: dictionary
        Background mean value of each channel for every time frame

    """

    X2R = x2-x1 #convert on steps because the rectangle patch definition
    Y2R = y2-y1

    #plot the defined area
    plt.figure(figsize=(8,8))
    fig = plt.gcf()
    ax = fig.gca()
    ax.imshow(data[channel][:,:,im_num])
    rect = matplotlib.patches.Rectangle((y1,x1), Y2R, X2R, linewidth=1, edgecolor='r', facecolor='none')
    ax.add_patch(rect)

    return()